# Importing Libraries

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
from cleverhans.tf2.attacks.projected_gradient_descent import projected_gradient_descent
import foolbox

# Adversarial Training

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images, test_images = train_images / 255.0, test_images / 255.0

In [ ]:
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
def adversarial_training(model, x, y, eps=0.3, alpha=0.01, epochs=10, batch_size=32):
    for _ in range(epochs):
        x_adversarial = projected_gradient_descent(model, x, eps=eps, eps_iter=alpha, nb_iter=1, norm=np.inf, clip_min=0.0, clip_max=1.0, targeted=False)
        x_combined = tf.concat([x, x_adversarial], axis=0)
        y_combined = tf.concat([y, y], axis=0)
        combined_dataset = tf.data.Dataset.from_tensor_slices((x_combined, y_combined)).shuffle(len(x_combined)).batch(batch_size)
        model.fit(combined_dataset, epochs=1, verbose=1)

In [ ]:
adversarial_training(model, train_images, train_labels)

3750/3750 [==============================] - 15s 4ms/step - loss: 0.0103 - accuracy: 0.9967


# Regularization

In [ ]:
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
adversarial_training(model, train_images, train_labels)

3750/3750 [==============================] - 14s 4ms/step - loss: 0.1083 - accuracy: 0.9648


# Ensemble Method

In [ ]:
def create_model():
    model = models.Sequential([
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

In [ ]:
num_models = 3
models = [create_model() for _ in range(num_models)]

for model in models:
    adversarial_training(model, train_images, train_labels)

In [ ]:
def ensemble_predict(models, x):
    predictions = np.zeros((len(x), 10))
    for model in models:
        predictions += model.predict(x)
    return np.argmax(predictions, axis=1)

In [ ]:
def evaluate_ensemble(models, x, y):
    predictions = ensemble_predict(models, x)
    accuracy = np.mean(predictions == y)
    return accuracy

In [ ]:
ensemble_accuracy = evaluate_ensemble(models, test_images, test_labels)
print("Ensemble Accuracy:", ensemble_accuracy)

313/313 [==============================] - 1s 2ms/step
Ensemble Accuracy: 0.9851


# Adding Noise to data

In [ ]:
train_images_noisy = train_images + np.random.normal(loc=0.0, scale=0.1, size=train_images.shape)
test_images_noisy = test_images + np.random.normal(loc=0.0, scale=0.1, size=test_images.shape)

In [ ]:
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
adversarial_training(model, train_images_noisy, train_labels)

3750/3750 [==============================] - 14s 4ms/step - loss: 0.0131 - accuracy: 0.9956


# Evaluating Model Robustness

In [ ]:
model = models.Sequential([
    layers.Flatten(input_shape=(28, 28)),
    layers.Dense(128, activation='relu'),
    layers.Dense(10, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
adversarial_training(model, train_images, train_labels)

In [ ]:
def evaluate_model(model, x, y):
    clean_accuracy = model.evaluate(x, y, verbose=0)[1]
    print("Clean Accuracy:", clean_accuracy)

    adv_fgsm = fast_gradient_method(model, x, 0.3, np.inf)
    fgsm_accuracy = model.evaluate(adv_fgsm, y, verbose=0)[1]
    print("FGSM Accuracy:", fgsm_accuracy)

    adv_pgd = projected_gradient_descent(model, x, eps=0.3, eps_iter=0.01, nb_iter=40, norm=np.inf, targeted=False)
    pgd_accuracy = model.evaluate(adv_pgd, y, verbose=0)[1]
    print("PGD Accuracy:", pgd_accuracy)

    fmodel = foolbox.models.TensorFlowModel(model, bounds=(0, 1))
    adv_deepfool = foolbox.attacks.L2DeepFoolAttack().run(fmodel, x, labels=y)

    deepfool_accuracy = model.evaluate(adv_deepfool, y, verbose=0)[1]
    print("DeepFool Accuracy:", deepfool_accuracy)

In [16]:
evaluate_model(model, test_images, test_labels)

Clean Accuracy: 0.9797000288963318
FGSM Accuracy: 0.010900000110268593
PGD Accuracy: 0.014999999664723873


TypeError: DeepFoolAttack.run() missing 1 required positional argument: 'criterion'